In [ ]:
pip install --upgrade yfinance

In [1]:
import yfinance as yf
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

# Scarica i dati storici da Yahoo Finance
symbol = 'AMZN'
data = yf.download(symbol, period="5d", interval="1m")

# Controlla se i dati sono stati scaricati correttamente
if data.empty:
    print("Nessun dato scaricato. Controlla il simbolo o la connessione.")
else:
    print("Dati scaricati con successo.")

print(data.head())

# Calcolo delle medie mobili a breve e lungo periodo
short_window = 10  # Periodo della media mobile corta (ad esempio, 10 minuti)
long_window = 30   # Periodo della media mobile lunga (ad esempio, 30 minuti)

data['Short_MA'] = data['Close'].rolling(window=short_window, min_periods=1).mean()
data['Long_MA'] = data['Close'].rolling(window=long_window, min_periods=1).mean()

# Identifica i segnali di "Gap" e "Volume"
gap_threshold = 0.005  # Gap > 0.5% tra apertura e chiusura precedente
volume_threshold = 1.5  # Volume maggiore di 1.5 volte la media degli ultimi 10 minuti

# Calcolare il gap come differenza tra apertura e chiusura precedente
data['Gap'] = (data['Open'] - data['Close'].shift(1)) / data['Close'].shift(1)

# Calcolare la media mobile del volume degli ultimi 10 minuti
data['Volume_MA'] = data['Volume'].rolling(window=10, min_periods=1).mean()

print("------- GAP --------")
print(data['Gap'])

# Rimuoviamo le righe che contengono NaN per la colonna 'Gap' (inclusa la prima riga)
data = data.dropna(subset=['Gap']) # Removing NaN values to align indexes

# Before calculating the Scalping_Signal, let's create the 'Position' column and initialize it to 0:
data['Position'] = 0

# Identificare le opportunità di scalping basate sul gap e sul volume
data['Scalping_Signal'] = 0  # Inizialmente nessun segnale
data['Scalping_Signal'] = np.where((data['Gap'] > gap_threshold) & (data['Volume'] > volume_threshold * data['Volume_MA']), 1, 0)

# Now the plotting code should work without the ValueError
plt.scatter(data.index[data['Position'] == 1], data['Close'][data['Position'] == 1], label='Buy Signal (MA)', marker='^', color='g', alpha=1)
plt.scatter(data.index[data['Position'] == -1], data['Close'][data['Position'] == -1], label='Sell Signal (MA)', marker='v', color='r', alpha=1)

# Creazione del grafico
plt.figure(figsize=(14, 8))

# Traccia il prezzo di chiusura
plt.plot(data['Close'], label='Close Price', color='blue', alpha=0.6)

# Traccia le medie mobili
plt.plot(data['Short_MA'], label=f'Short {short_window}-minute MA', color='red', alpha=0.75)
plt.plot(data['Long_MA'], label=f'Long {long_window}-minute MA', color='green', alpha=0.75)

# Aggiungi segnali di Buy e Sell basati sulle medie mobili
plt.scatter(data.index[data['Position'] == 1], data['Close'][data['Position'] == 1], label='Buy Signal (MA)', marker='^', color='g', alpha=1)
plt.scatter(data.index[data['Position'] == -1], data['Close'][data['Position'] == -1], label='Sell Signal (MA)', marker='v', color='r', alpha=1)

# Aggiungi segnali di Buy basati su Gap e Volume
plt.scatter(data.index[data['Scalping_Signal'] == 1], data['Close'][data['Scalping_Signal'] == 1], label='Buy Signal (Gap + Volume)', marker='*', color='orange', alpha=1)

# Aggiungi titolo e etichette
plt.title(f'{symbol} Scalping Opportunities (6 Months)')
plt.xlabel('Date')
plt.ylabel('Price (USD)')
plt.legend(loc='best')
plt.grid(True)
plt.xticks(rotation=45)
plt.tight_layout()

# Mostra il grafico
plt.show()

[*********************100%***********************]  1 of 1 completed


Dati scaricati con successo.
Price                       Adj Close       Close        High         Low  \
Ticker                           AMZN        AMZN        AMZN        AMZN   
Datetime                                                                    
2025-01-08 14:30:00+00:00  222.610001  222.610001  223.164993  222.610001   
2025-01-08 14:31:00+00:00  222.690002  222.690002  222.759995  222.250000   
2025-01-08 14:32:00+00:00  222.330002  222.330002  222.838501  222.130005   
2025-01-08 14:33:00+00:00  222.229996  222.229996  222.520004  222.214996   
2025-01-08 14:34:00+00:00  222.725006  222.725006  222.850006  222.225906   

Price                            Open   Volume  
Ticker                           AMZN     AMZN  
Datetime                                        
2025-01-08 14:30:00+00:00  223.149994  1066708  
2025-01-08 14:31:00+00:00  222.660004    95117  
2025-01-08 14:32:00+00:00  222.725006   100055  
2025-01-08 14:33:00+00:00  222.360001    58351  
2025-01-08 

KeyError: ['Gap']